In [1]:
from aglaia import symm_funct
from aglaia.tests import np_symm_funct
import numpy as np
import tensorflow as tf

/Users/walfits/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
input_data = "/Volumes/Transcend/repositories/Aglaia/aglaia/tests/qm7_testdata.npz"
data = np.load(input_data)

In [3]:
xyzs = data["arr_0"]
zs = data["arr_1"]
elements = data["arr_2"]
element_pairs = data["arr_3"]

In [4]:
n_samples = zs.shape[0]
max_n_atoms = zs.shape[1]

In [5]:
zs_tf = tf.placeholder(shape=[n_samples, max_n_atoms], dtype=tf.int32, name="zs")
xyz_tf = tf.placeholder(shape=[n_samples, max_n_atoms, 3], dtype=tf.float32, name="xyz")

In [6]:
radial_cutoff = 500.0
angular_cutoff = 500.0
radial_rs = [0.0]
angular_rs = [0.0]
theta_s_ar = [3.0]
zeta = 3.0
eta = 2.0

In [7]:
rad_cutoff = tf.constant(radial_cutoff, dtype=tf.float32)
ang_cutoff = tf.constant(angular_cutoff, dtype=tf.float32)
rad_rs = tf.constant(radial_rs, dtype=tf.float32)
ang_rs = tf.constant(angular_rs, dtype=tf.float32)
theta_s = tf.constant(theta_s_ar, dtype=tf.float32)
zeta_tf = tf.constant(zeta, dtype=tf.float32)
eta_tf = tf.constant(eta, dtype=tf.float32)

In [8]:
pre_sum_rad = symm_funct.acsf_rad(xyz_tf, zs_tf, rad_cutoff, rad_rs, eta_tf)  # (n_samples, n_atoms, n_atoms, n_rad_rs)

In [9]:
rad_term = symm_funct.sum_rad(pre_sum_rad, zs_tf, elements, rad_rs)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
tf_rad = sess.run(rad_term, feed_dict={xyz_tf: xyzs, zs_tf:zs})

In [12]:
rad_term = np_symm_funct.acsf_rad(xyzs, zs, elements, radial_cutoff, radial_rs, eta)

In [13]:
pre_sum_ang = symm_funct.acsf_ang(xyz_tf, zs_tf, ang_cutoff, ang_rs, theta_s, zeta_tf, eta_tf)

In [14]:
ang_term = symm_funct.sum_ang(pre_sum_ang, zs_tf, element_pairs, ang_rs, theta_s)

In [15]:
tf_ang = sess.run(ang_term, feed_dict={xyz_tf: xyzs, zs_tf:zs})

In [16]:
# tf_ang = sess.run(pre_sum_ang, feed_dict={xyz_tf: xyzs, zs_tf:zs})

In [17]:
ang_term = np_symm_funct.acsf_ang(xyzs, zs, element_pairs, angular_cutoff, angular_rs, theta_s_ar, zeta, eta)

In [18]:
# print(np.any(np.isnan(tf_ang[1])))
# print(np.any(tf_ang[1]<-1))

In [19]:
# print(np.any(np.isnan(tf_ang[2])))
# print(np.any(tf_ang[2]==0))

In [20]:
print(tf_ang)

[[[4.37901467e-01 0.00000000e+00 0.00000000e+00]
  [3.30623618e-04 0.00000000e+00 2.50179670e-04]
  [3.30630282e-04 0.00000000e+00 2.50183075e-04]
  [3.30633833e-04 0.00000000e+00 2.50184210e-04]
  [3.30635521e-04 0.00000000e+00 2.50185782e-04]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[2.39127696e-01 0.00000000e+00 8.24202448e-02]
  [2.39127815e-01 0.00000000e+00 8.24202523e-02]
  [2.23400348e-04 4.82145879e-05 4.37797600e-04]
  [2.23390714e-04 4.82144569e-05 4.37777431e-04]
  [2.23442359e-04 4.82373798e-05 4.37895535e-04]
  [2.23441763e-04 4.82373107e-05 4.37893817e-04]
  [2.23392664e-04 4.82144715e-05 4.37779614e-04]
  [2.23399809e-04 4.82147479e-05 4.37797833e-04]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[1.19420528e-01 0.00000000e+00 1.21512234e-01]
  [1.19420536e-01 0.00000000e+00 1.21512257e-01]
  [3.48050307e-0

In [21]:
nan_idx = np.argwhere(np.isnan(tf_ang[2]))
print(nan_idx)

[]


In [22]:
print(tf_ang[1][nan_idx])

[]
